In [65]:
import os, requests, base64, imageio
from PIL import Image
from io import BytesIO
import seaborn as sn
import pandas as pd
import numpy as np
from scipy import misc
import re
import glob
import matplotlib.pyplot as plt
import skimage
import skimage.io
import skimage.viewer

C:\Users\Juanathan\anaconda3\lib\site-packages\skimage\viewer\utils\__init__.py:1: UserWarning: Recommended matplotlib backend is `Agg` for full skimage.viewer functionality.
  from .core import *


In [67]:
# funtions
def topo_change(image1, image2):
    
    num_idx = re.compile(r'\d+')
    name = re.split(r'_',image1)
    
    if len(name) > 5:
        city = name[1]+ ' ' + name[2]
    else:
        city = name[1]
    
    idx1 = num_idx.findall(image1)
    year1 = int(idx1[1])
    
    idx2 = num_idx.findall(image2)
    year2 = int(idx2[2])

    #im1 = imageio.imread(image1)
    #print(im1.shape)
    
    avg_rgb_1 = imageio.imread(image1).mean((0,1))
    avg_rgb_2 = imageio.imread(image2).mean((0,1))
    
    per_diff = abs(avg_rgb_1-avg_rgb_2) / avg_rgb_1
    delta_year = abs(year2 - year1)
    
#get histogram values for both images
    print(image1)
    print(image2)
    name1 = os.path.basename(image1)
    name2 = os.path.basename(image2)
    print(name1)
    print(name2)
    
    img1 = Image.open('images/'+name1)
    img2 = Image.open('images/'+name1)
    
    #iterate through list and attach read image to capitals dataframe
    bef_hist = skimage.io.imread('images/'+name1)
    aft_hist = skimage.io.imread('images/'+name2)
#     bef_hist = 'images/'+name1
#     aft_hist = 'images/'+name2
    
    
    return city, (per_diff/delta_year)*100, bef_hist, aft_hist

# TESTING #    
PATH = os.path.join(os.getcwd(),'images')

image1 = os.path.join(PATH, '_Albany_2017_03_01.png')
image2 = os.path.join(PATH, '_Albany_2020_05_01.png')

city, rate_of_change, bef_hist, aft_hist = topo_change(image1, image2)

print("FUNCTION TESTING:")
print("The rate of change between year 2017 and 2020 for Albany, New York\n R:%.5f percent, G:%.5f percent, B:%.5f percent"
                                            %(rate_of_change[0], rate_of_change[1], rate_of_change[2]))
print(bef_hist)
print(aft_hist)

C:\Users\Juanathan\SatteliteAnalysis\images\_Albany_2017_03_01.png
C:\Users\Juanathan\SatteliteAnalysis\images\_Albany_2020_05_01.png
_Albany_2017_03_01.png
_Albany_2020_05_01.png
FUNCTION TESTING:
The rate of change between year 2017 and 2020 for Albany, New York
 R:6.67396 percent, G:6.88094 percent, B:11.14665 percent
[[[ 32  36  45]
  [ 55  61  73]
  [ 49  56  68]
  ...
  [ 33  37  50]
  [ 34  39  53]
  [ 30  34  45]]

 [[ 37  42  55]
  [ 45  52  65]
  [ 41  45  55]
  ...
  [ 24  27  37]
  [ 29  32  44]
  [ 30  33  46]]

 [[ 30  34  46]
  [ 33  37  50]
  [ 41  47  60]
  ...
  [ 26  29  38]
  [ 32  36  47]
  [ 32  36  49]]

 ...

 [[ 27  30  41]
  [ 24  28  39]
  [ 25  29  40]
  ...
  [ 23  29  41]
  [  9  12  19]
  [ 40  48  63]]

 [[ 26  30  41]
  [ 26  30  41]
  [ 33  39  54]
  ...
  [ 17  20  29]
  [ 55  63  76]
  [ 49  58  73]]

 [[ 24  28  41]
  [ 29  34  47]
  [ 33  39  53]
  ...
  [ 24  30  42]
  [ 83  95 113]
  [ 53  59  70]]]
[[[ 39  44  58]
  [ 66  75  92]
  [ 56  67  87]

In [ ]:
# Data Set Cleaning

all_cities = pd.read_csv('us_cities_v2.csv')
all_cities.head()

capital_mask = all_cities['capital'] == True
just_capitals = all_cities[capital_mask]

just_capitals = just_capitals.sort_values(by=['city'])
drop_list = ['state', 'military', 'incorporated'] 

just_capitals
# we might add state back into our final data set if we cannot remember state id's
# removed 'military' because all were 'False'
# removed 'incorporated' becuase all were 'True'

just_capitals=just_capitals.drop(columns=drop_list)

just_capitals=just_capitals.reset_index(drop=True)

empty = []
path = []
cities = []
data = []

# loop over png images in folder ~/images
i = 0
for image in sorted(glob.glob(os.path.join(PATH,'*.png'))):
    
    name = re.split(r'_',image)
    
    if len(name) > 5:
        city = name[1]+ ' ' + name[2]
    else:
        city = name[1]
    
    path.append(image)
    empty.append(city)

    if i > 0:
        
        if empty[i] == empty[i-1]:
            
            image1 = os.path.join(path[i])
            image2 = os.path.join(path[i-1])
            print(image1)
            print(image2)
            city, rate_of_change, bef_hist, aft_hist = topo_change(image1, image2)
            data.append([city, rate_of_change[0], rate_of_change[1], rate_of_change[2], bef_hist, aft_hist])
  
    i += 1

df = pd.DataFrame(data, columns = ['Cities', 'dR_year', 'dG_year', 'dB_year', 'bef_hist', 'aft_hist'])

just_capitals['dR_year'] = df['dR_year'] 
just_capitals['dG_year'] = df['dG_year'] 
just_capitals['dB_year'] = df['dB_year']


just_capitals = just_capitals.drop(columns='capital')
just_capitals.to_csv('just_capitals.csv')

just_capitals
df.to_csv('color_histograms.csv')

C:\Users\Juanathan\SatteliteAnalysis\images\_Albany_2020_05_01.png
C:\Users\Juanathan\SatteliteAnalysis\images\_Albany_2017_03_01.png
C:\Users\Juanathan\SatteliteAnalysis\images\_Albany_2020_05_01.png
C:\Users\Juanathan\SatteliteAnalysis\images\_Albany_2017_03_01.png
_Albany_2020_05_01.png
_Albany_2017_03_01.png
C:\Users\Juanathan\SatteliteAnalysis\images\_Annapolis_2019_06_01.png
C:\Users\Juanathan\SatteliteAnalysis\images\_Annapolis_2017_08_01.png
C:\Users\Juanathan\SatteliteAnalysis\images\_Annapolis_2019_06_01.png
C:\Users\Juanathan\SatteliteAnalysis\images\_Annapolis_2017_08_01.png
_Annapolis_2019_06_01.png
_Annapolis_2017_08_01.png
C:\Users\Juanathan\SatteliteAnalysis\images\_Atlanta_2017_03_01.png
C:\Users\Juanathan\SatteliteAnalysis\images\_Atlanta_2016_05_01.png
C:\Users\Juanathan\SatteliteAnalysis\images\_Atlanta_2017_03_01.png
C:\Users\Juanathan\SatteliteAnalysis\images\_Atlanta_2016_05_01.png
_Atlanta_2017_03_01.png
_Atlanta_2016_05_01.png
C:\Users\Juanathan\SatteliteAnalys

NameError: name 'df' is not defined